In [4]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

import requests
import datetime

from requests.auth import HTTPBasicAuth

import json
import pandas as pd
from pandas.io.json import json_normalize

In [5]:
### PORTFOLIO: GET SUBSIDIARIES AND THE RELATIVE GUID ###
bitsight_api_key = 'f6780e181b835bf09491cfcaeaeacee5ab8a8c87'
bitsight_portfolio_url = "https://api.bitsighttech.com/ratings/v2/portfolio"

try:
    res = requests.post(url=bitsight_portfolio_url, verify=True, auth = HTTPBasicAuth(bitsight_api_key, ''))
    res_code = res.status_code 
    if (res_code != 200):
        print("HTTP Error: ", res_code)

except requests.exceptions.SSLError as ssl_error:
    inner_exception = ssl_error.args[0]
    inner_ssl_error = inner_exception.reason.args[0]
    if str(inner_ssl_error).find('SSL: CERTIFICATE_VERIFY_FAILED') == -1:
        for key, value in vars(inner_ssl_error).items():
            dnshost = value['subject'][0][0][1]
            is_ssl_error = True
            for alt in value['subjectAltName']:
                if alt[0]=="DNS":
                    san.append(alt[1])
                pass
            
except Exception as e:
    print(type(e))
    print(e)
                
except requests.exceptions.RequestException as e:
    print("Error: ", e)
    pass
            
except urllib3.exceptions as e:
    print("Error: ", e)
    pass
            
except socket.gaierror as e:
    print("Error: ", e)
    pass

In [6]:
jsonResponse = res.json()
df1 = pd.json_normalize(
    jsonResponse,
    record_path=[['results'][0]])

#df1 = df1.rename(columns={'agent_hostname':'endpoint_name'})

In [7]:
jsonResponse

{'links': {'next': None, 'previous': None},
 'count': 13,
 'summaries': {'my-company': '61622d70-ac9b-49d9-a1d9-05025e2cd237'},
 'results': [{'guid': 'ec8c38ba-30d1-4bf4-8174-af4db9bca3c1',
   'custom_id': None,
   'name': 'Bank of America Corporation',
   'shortname': 'Bank of America',
   'network_size_v4': 7088434,
   'rating': 770,
   'rating_date': '2022-02-08',
   'added_date': '2020-02-28',
   'industry': {'name': 'Finance', 'slug': 'finance'},
   'sub_industry': {'name': 'Banking', 'slug': 'banking'},
   'type': ['CURATED'],
   'logo': 'https://api.bitsighttech.com/ratings/v1/companies/ec8c38ba-30d1-4bf4-8174-af4db9bca3c1/logo-image',
   'sparkline': 'https://api.bitsighttech.com/ratings/v1/companies/ec8c38ba-30d1-4bf4-8174-af4db9bca3c1/sparkline?size=small',
   'subscription_type': {'name': 'Total Risk Monitoring',
    'slug': 'continuous_monitoring'},
   'primary_domain': 'bankofamerica.com',
   'display_url': 'https://service.bitsighttech.com/app/company/ec8c38ba-30d1-4bf4-8

In [11]:
subsidiaries = pd.DataFrame(df1, columns=['name', 'guid', 'rating', 'rating_date'])
subsidiaries.set_index('name', inplace=True)
subsidiaries.drop('Kering, Inc - Luxury Brands', inplace=True)
subsidiaries.drop('Bank of America Corporation', inplace=True)
subsidiaries.drop('SurveyMonkey Corporation', inplace=True)
subsidiaries

# entries to remove
#"Bank of America Corporation"
#"Kering, Inc - Luxury Brands"
#"SurveyMonkey Corporation"

,guid,rating,rating_date
name,,,
Essilor International Group,bfc11446-2f48-4bff-8e1d-6443f9a4e8cc,530,2022-02-08
EyeMed Vision Care LLC,4121709c-bb1b-44d2-82da-f53f7df3cf30,730,2022-02-08
Luxottica Americas,3c3916a1-1e50-4f14-8c62-3273b16c3658,590,2022-02-08
Luxottica APAC,9af338f9-c238-45d5-b829-89a028ecd66c,740,2022-02-08
Luxottica Australia,e4590952-53e9-4635-a1ae-ee27dfe09795,780,2022-02-08
Luxottica Brazil,3673e0d3-5634-47b8-9c61-2b78d768b88b,760,2022-02-08
Luxottica Bundle,5f81df2d-e3c7-4ed1-af94-c643c26374e6,610,2022-02-08
Luxottica EMEA,c6c8c8ae-0d4b-4142-8d69-347f792b96de,740,2022-02-08
Luxottica Group - Core Network,a633d496-54c9-40f0-88db-a5117453afe0,670,2022-02-08


In [20]:
# DF that will be saved as CSV at the end of the iteration
df_vulns = pd.DataFrame(columns=['asset', 'port', 'CVE', 'CVSS','confidence'])
    
# get_findings(): Get the bitsight findings in a json format, using requests lib
# parameter 'sub': GUID of the subsidiary (got from the portfolio)
def get_findings(sub, df_vulns):

    bitsight_findings_url = "https://api.bitsighttech.com/v1/companies/" + sub + "/findings"

    try:
        res = requests.post(url=bitsight_findings_url, verify=True, auth = HTTPBasicAuth(bitsight_api_key, ''))
        res_code = res.status_code 
        if (res_code != 200):
            print("HTTP Error: ", res_code)

        jsonResponse = res.json()
        print("Count: ",jsonResponse['count'])
        print(bitsight_findings_url)
        df_vulns = get_vulns(jsonResponse, df_vulns)
        bitsight_findings_url_next = jsonResponse['links']['next']

        # while loop to iterate over the findings (if more than 100)
        # get the vulnerabilities and append into the DF get_vulns
        while bitsight_findings_url_next is not None:    

            print(bitsight_findings_url_next)
            try:
                res = requests.post(url=bitsight_findings_url_next, verify=True, auth = HTTPBasicAuth(bitsight_api_key, ''))
                res_code = res.status_code 
                if (res_code != 200):
                    print("HTTP Error: ", res_code)

                jsonResponse = res.json()
                df_vulns = get_vulns(jsonResponse, df_vulns)
                bitsight_findings_url_next = jsonResponse['links']['next']
            
            except Exception as e:
                print(type(e))
                print(e)

    except Exception as e:
        print(type(e))
        print(e)
            
# get_vulns(): Get the vulnerabilities from a findings page, append it to the DF vuln_report
# parameter 'jsonResponse': json page of findings (default limit/offset is 100)
# parameter 'vuln_report': DF used to store all vulnerabilities related to a subsidiary 
def get_vulns(jsonResponse,df_vulns):
    iloc = len(df_vulns)
    x = 0
        
    while x < len(jsonResponse['results']):  
        for asset in jsonResponse['results'][x]['assets']:
            if "vulnerabilities" in jsonResponse['results'][x]['details']:
                for vuln in jsonResponse['results'][x]['details']['vulnerabilities']:
                    if "dest_port" in jsonResponse['results'][x]['details']:
                        row = [asset['asset'],jsonResponse['results'][x]['details']['dest_port'],vuln['name'],vuln['cvss'],vuln['confidence']]
                    else:
                        row = [asset['asset'],'',vuln['name'],vuln['cvss'],vuln['confidence']]
                    df_vulns.loc[iloc] = row
                    iloc = iloc +1
        x = x +1
    
    return df_vulns

# foreach subsidiary in the portfolio: 
# 1/ get the vulns from the finding (while loop to pagination)
# 2/ save it in a dedicated CSV
for index,sub in subsidiaries['guid'].iteritems():

    df_vulns_report = pd.DataFrame(columns=['asset', 'port', 'CVE', 'CVSS','confidence'])
    get_findings(sub, df_vulns_report)
    
    # write the DF get_vulns in a CSV 
    pd.set_option('display.max_colwidth', None)            
    current_date = datetime.datetime.now()
    cdate = str(current_date.year) + str(current_date.month) + str(current_date.day)
    input_file = "bitsight_" +  cdate +"_"+ sub  
    df_vulns_report.to_csv(str(input_file + '.csv'))

Count:  3138
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=100
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=200
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=300
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=400
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=500
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=600
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=700
https://api.bitsighttech.com/v1/companies/bfc11446-2f48-4bff-8e1d-6443f9a4e8cc/findings?limit=100&offset=800
https://api.bitsighttech.co

https://api.bitsighttech.com/v1/companies/9af338f9-c238-45d5-b829-89a028ecd66c/findings?limit=100&offset=1200
https://api.bitsighttech.com/v1/companies/9af338f9-c238-45d5-b829-89a028ecd66c/findings?limit=100&offset=1300
https://api.bitsighttech.com/v1/companies/9af338f9-c238-45d5-b829-89a028ecd66c/findings?limit=100&offset=1400
https://api.bitsighttech.com/v1/companies/9af338f9-c238-45d5-b829-89a028ecd66c/findings?limit=100&offset=1500
Count:  243
https://api.bitsighttech.com/v1/companies/e4590952-53e9-4635-a1ae-ee27dfe09795/findings
https://api.bitsighttech.com/v1/companies/e4590952-53e9-4635-a1ae-ee27dfe09795/findings?limit=100&offset=100
https://api.bitsighttech.com/v1/companies/e4590952-53e9-4635-a1ae-ee27dfe09795/findings?limit=100&offset=200
Count:  340
https://api.bitsighttech.com/v1/companies/3673e0d3-5634-47b8-9c61-2b78d768b88b/findings
https://api.bitsighttech.com/v1/companies/3673e0d3-5634-47b8-9c61-2b78d768b88b/findings?limit=100&offset=100
https://api.bitsighttech.com/v1/c

https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=6400
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=6500
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=6600
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=6700
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=6800
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=6900
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=7000
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=7100
https://api.bitsighttech.com/v1/companies/5f81df2d-e3c7-4ed1-af94-c643c26374e6/findings?limit=100&offset=7200
https://ap

https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1200
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1300
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1400
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1500
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1600
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1700
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1800
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=1900
https://api.bitsighttech.com/v1/companies/a633d496-54c9-40f0-88db-a5117453afe0/findings?limit=100&offset=2000
https://ap

https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1200
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1300
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1400
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1500
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1600
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1700
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1800
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=1900
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=2000
https://ap

https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=8700
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=8800
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=8900
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=9000
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=9100
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=9200
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=9300
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=9400
https://api.bitsighttech.com/v1/companies/61622d70-ac9b-49d9-a1d9-05025e2cd237/findings?limit=100&offset=9500
https://ap

In [35]:
jsonResponse = res.json()
jsonResponse

{'links': {'next': None, 'previous': None},
 'count': 13,
 'summaries': {'my-company': '61622d70-ac9b-49d9-a1d9-05025e2cd237'},
 'results': [{'guid': 'ec8c38ba-30d1-4bf4-8174-af4db9bca3c1',
   'custom_id': None,
   'name': 'Bank of America Corporation',
   'shortname': 'Bank of America',
   'network_size_v4': 7088434,
   'rating': 770,
   'rating_date': '2022-02-07',
   'added_date': '2020-02-28',
   'industry': {'name': 'Finance', 'slug': 'finance'},
   'sub_industry': {'name': 'Banking', 'slug': 'banking'},
   'type': ['CURATED'],
   'logo': 'https://api.bitsighttech.com/ratings/v1/companies/ec8c38ba-30d1-4bf4-8174-af4db9bca3c1/logo-image',
   'sparkline': 'https://api.bitsighttech.com/ratings/v1/companies/ec8c38ba-30d1-4bf4-8174-af4db9bca3c1/sparkline?size=small',
   'subscription_type': {'name': 'Total Risk Monitoring',
    'slug': 'continuous_monitoring'},
   'primary_domain': 'bankofamerica.com',
   'display_url': 'https://service.bitsighttech.com/app/company/ec8c38ba-30d1-4bf4-8

In [13]:
### TESTS FOR ONE SUB ###
company_guid = "bfc11446-2f48-4bff-8e1d-6443f9a4e8cc"
bitsight_findings = "https://api.bitsighttech.com/v1/companies/" + company_guid + "/findings"

try:
    res = requests.post(url=bitsight_findings, verify=True, auth = HTTPBasicAuth(bitsight_api_key, ''))
    res_code = res.status_code 
    if (res_code != 200):
        print("HTTP Error: ", res_code)

except requests.exceptions.SSLError as ssl_error:
    inner_exception = ssl_error.args[0]
    inner_ssl_error = inner_exception.reason.args[0]
    if str(inner_ssl_error).find('SSL: CERTIFICATE_VERIFY_FAILED') == -1:
        for key, value in vars(inner_ssl_error).items():
            dnshost = value['subject'][0][0][1]
            is_ssl_error = True
            for alt in value['subjectAltName']:
                if alt[0]=="DNS":
                    san.append(alt[1])
                pass
            
except Exception as e:
    print(type(e))
    print(e)
                
except requests.exceptions.RequestException as e:
    print("Error: ", e)
    pass
            
except urllib3.exceptions as e:
    print("Error: ", e)
    pass
            
except socket.gaierror as e:
    print("Error: ", e)
    pass

In [36]:
### TESTS FOR A SINGLE SUB ###
vuln_report = pd.DataFrame(columns=['asset', 'port', 'CVE', 'CVSS','confidence'])
iloc = 0

for finding in jsonResponse['results']:   
    for asset in finding['assets']:
        if "vulnerabilities" in finding['details']:
            for vuln in finding['details']['vulnerabilities']:
                row = [asset['asset'],finding['details']['dest_port'],vuln['name'],vuln['cvss'],vuln['confidence']]
                vuln_report.loc[iloc] = row
                iloc = iloc +1

vuln_report

KeyError: 'assets'